### Testing the extraction parameters

In [1]:
using PyCall
using Statistics , StatsBase
using  DataFrames, Printf

import PyPlot
sns= pyimport("seaborn")

## directory
rootdir = ENV["GAIA_ROOT"]
wdir    = "$rootdir/products"
votdir  = "$rootdir/products/votable"
plotdir = "$rootdir/products/testfull2"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering


cd(wdir)

In [2]:
mutable struct _extractParam
    rootdir::String
    wdir::String
    votdir::String
    plotdir::String
    ocdir::String

    minQ::Float64
    minstars::Int
    maxstars::Int
    forcedminstars::Int
    mingoodsolution::Int
    niterqminq::Int
    nburnout::Int
    niter::Int

    epsmean::Float64
    epsdisp::Float64
    min_nei::Int
    min_cl::Int
    ncoredisp::Int
    w3dmean::Float64
    w3ddisp::Float64
    wvelmean::Float64
    wveldisp::Float64
    whrdmean::Float64
    whrddisp::Float64

    aperture2d::Float64
    maxaperture2d::Float64 
    aperture3d::Float64    
    maxaperture3d::Float64
    aperturev::Float64   
    maxaperturev::Float64
    nboot::Int
    labels::String
    
    _extractParam()= new()
end

## return the default parameters
##
function _set_default_params()::_extractParam
    def = _extractParam()
    
    ## Directories
    def.rootdir =  ENV["GAIA_ROOT"]
    def.wdir    = "$rootdir/e2e_products"
    def.votdir  = "$wdir/votable"
    def.plotdir = "$wdir/plots-select"
    def.ocdir   = "$wdir/ocfull"

    ## MCMC
    ##
    def.minQ    = 2.7
    def.minstars = 40
    def.maxstars = 10000
    def.forcedminstars = 30
    def.mingoodsolution = 10        # minimum good solution for Q in check_qminqstar
    def.niterqminq = 500            # check_qminqstar..

    def.nburnout  = 2000            # burn-in iterations
    def.niter     = 15000           # MCMC iterations

    ## prior settings
    ##
    def.epsmean   = 2.5             # epsilon mean
    def.epsdisp   = 1.5             # epsilon dispersion
    def.min_nei   = 10              # min_nei mean
    def.min_cl    = 15              # min_cl mean
    def.ncoredisp = 10              # dispersion for min_nei and min_cl
    def.w3dmean   = 6.0
    def.w3ddisp   = 4.0
    def.wvelmean  = 6.0
    def.wveldisp  = 4.0
    def.whrdmean  = 2.0
    def.whrddisp  = 1.5

    ## Metrics find_clusters
    def.aperture2d    = 1.5
    def.maxaperture2d = 15
    def.aperture3d    = 3
    def.maxaperture3d = 20
    def.aperturev     = 3.0
    def.maxaperturev  = 20
    def.nboot         = 50          # bootstrap number

    def.labels        = "Qc"        # method to get labels: Qc|Qn
    
    return(def)
end

_set_default_params (generic function with 1 method)

In [3]:
function _read_params(file)::_extractParam
end

_read_params (generic function with 1 method)

In [4]:
## Main
function main(file)
    
    default= _set_default_params()
    println(default)
end

main (generic function with 1 method)

In [5]:
file= "default.ext"
main(file)

_extractParam("/home/stephane/Science/GAIA", "/home/stephane/Science/GAIA/e2e_products", "/home/stephane/Science/GAIA/products/votable", "/home/stephane/Science/GAIA/products/plots-select", "/home/stephane/Science/GAIA/products/ocfull", 2.7, 40, 10000, 30, 10, 500, 2000, 15000, 2.5, 1.5, 10, 15, 10, 6.0, 4.0, 6.0, 4.0, 2.0, 1.5, 1.5, 15.0, 3.0, 20.0, 3.0, 20.0, 50, "Qc")
